## Verificación de locutores

Dado un segmento de voz Y, y un locutor hipotético S, la tarea de la verificación de locutores es determinar si Y fue dicho por S.  
Supondremos siempre que Y siempre tiene locuciones de un solo locutor.

La verificación de locutores puede ser planteada como un test de hipótesis tal que:

$H_0$: Y pertenece al locutor hipotético S 

O bien:

$H_1$: Y no pertenece al locutor hipotético S

La forma óptima de determinar qué hipótesis es verdadera es utilizando el likelihood ratio dado por: 

$$
\frac{p(Y|H_0)}{p(Y|H_1)}=\left\{
        \begin{array}{ll}
            \geq \theta & \quad \textrm{aceptar }  H_0 \\
            < \theta & \quad \textrm{rechazar }  H_0
        \end{array}
    \right.
$$

Donde $\theta$ es el umbral de decisión.

El módulo de front-end se encarga de realizar la extracción de características (features) de la señal de voz. Los modelos a utilizar deben estar representados en función de estos features. Un caso particular de estos features son los coeficientes MFCC ya que están vinculados al modelo de la cóclea, la cual está adaptada particularmente para interpretar la voz humana. Otra posibilidad que se utiliza para verificación de locutores son los coeficientes LPC los cuales contienen información acerca de como la señal es generada por el aparato fonador. Adicionalmente, el módulo de front-end puede contener un VAD, un supresor de ruido y un filtro de preénfasis. El VAD es muy útil ya que permite que uno no tenga en consideración muchas muestras que aportarían ceros generando un corrimiento de los parámetros estadísticos. Además para nuestro caso, el silencio forma parte de la mayoría de las locuciones, lo cual atenta en contra de la discriminación entre características. Los valores del MFCCs no son muy robustos ante la presencia de ruido aditivo, por ello es común que se implemente una etapa de supresión de ruido, o ligeras modificaciones de MFCC. Por último la señal de voz contiene información en sus altas frecuencias (las cuales tienen tasas de variación mas altas) que las señales de bajas frecuencias. Es por ello que se suele considerar un filtro de preénfasis. El filtro de preénfasis mas utilizado tiene la forma: 

$$ y(n)=x(n)-\alpha.x(n-1), \textrm{ con }0.9 < \alpha < 0.97 $$

Una pregunta que puede surgir en este momento es cuál es la conveniencia de utilizar MFCC (con la DCT en el bloque final del proceso) en vez de utilizar un banco de filtros logarítmico. Para responder esto, ya vamos adelantando que la forma de modelar tanto al modelo de mundo como al modelo del locutor se hará utilizando GMM. GMM mejora mucho su performance si cada una de sus variables es independiente. En general se observa que la etapa de DCT elimina dependencia entre los coeficientes MFCC, mejorando el rendimiento del sistema. Esta es una de las razones de peso por las cuales MFCC tiene una DCT a la salida y no una IFFT.

Para mas información con respecto a la generación de los coeficientes MFCC se recomienda ver este [link](http://www.practicalcryptography.com/miscellaneous/machine-learning/guide-mel-frequency-cepstral-coefficients-mfccs/)

Algunas consideraciones sobre el paquete [python_speech_features](http://python-speech-features.readthedocs.io/en/latest/index.html#):

* Ya aplica filtro de preenfasis
* si bien calcua los 26 MFCC solo devuelve los primeros 13, los cuales son los mas importantes para la verificación de locutores.
* appendEnergy pone en el bin 1 de los features el logaritmo de la energía. Esto ocurre ya que en general se descartan los 2 o 3 primeros elementos de los MFCC ya que no se ve un aumento significativo del score y sí se ve un aumento en el score si agrego a mi modelo el logaritmo de la energía del frame. Para mas información se recomienda leer este [paper](http://ai2-s2-pdfs.s3.amazonaws.com/6f14/5968059d006235d59d1c201d5185b440d221.pdf).

Adicionalmente se suelen aplicar algunos liftros y los features conocidos como delta y delta-delta como veremos mas adelante.

In [1]:
from python_speech_features import mfcc 
from python_speech_features import logfbank
from python_speech_features import base
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
import numpy as np
from sklearn import mixture
import itertools
from scipy import linalg
import matplotlib as mpl

def mfcc_wav(file):
    (rate,sig) = wav.read(file)
    mfcc_feat = mfcc(sig,rate,nfft=512,appendEnergy=True)
    #fbank_feat = logfbank(sig,rate,nfft=512)
    return mfcc_feat

mfcc_feat=mfcc_wav("mundo.wav")
plt.hist(mfcc_feat.T[5], bins=40)
plt.title("Histogram")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

<Figure size 640x480 with 1 Axes>

In [2]:
WINDOW_SIZE=20
mundo_size=mfcc_feat.shape[0]-WINDOW_SIZE+1
mundo=np.array([mfcc_feat[i:i+WINDOW_SIZE].reshape(1,WINDOW_SIZE*mfcc_feat.shape[1])[0] for i in range(mundo_size)])

In [3]:
mundo[1]

array([  8.2735939 , -36.54609875,  23.69603008, -16.5070533 ,
        12.01870915, -18.15439091,  15.74576462,  -6.73881606,
        -3.24585288,   4.89165966,  -4.15684418,  -0.8209452 ,
        -1.62943819,   7.98226232, -46.13965039,  22.97692025,
       -10.66959535,   7.71275206, -22.58557751,  18.74388904,
        -3.1929766 ,  -4.68366538,   3.15471241,  -3.19703806,
        -0.45735365,   1.31570582,   8.04361292, -39.20014361,
        21.69446529, -13.86175807,   9.80813936, -20.22351231,
        14.31072568,  -7.7443988 ,  -4.76882842,   2.23613116,
        -3.91457845,  -0.24096122,  -1.43206452,   8.01514461,
       -43.6777627 ,  23.73037114, -11.21375175,   9.00604438,
       -18.17757488,  18.03384752,  -3.43744575,  -3.70497148,
         2.55588557,  -5.34041547,   4.52767307,   0.90422581,
         8.0651455 , -41.38219654,  22.57875537, -11.47967352,
         9.02568296, -18.53798189,  16.77933841,  -6.70160264,
        -5.40524852,   3.99116116,  -5.36557172,   2.21

In [4]:
mundo.shape

(8999, 260)

In [5]:
mundo_labels=np.array([[0,1]]*mundo.shape[0])

In [6]:
mundo_labels[0]

array([0, 1])

In [7]:
mundo_labels.shape

(8999, 2)

In [8]:
mfcc_feat=mfcc_wav("homero.wav")

In [9]:
homero_size=mfcc_feat.shape[0]-WINDOW_SIZE+1
homero=np.array([mfcc_feat[i:i+WINDOW_SIZE].reshape(1,WINDOW_SIZE*mfcc_feat.shape[1])[0] for i in range(homero_size)])

In [10]:
homero[0]

array([ 13.42441896,  -9.54737569,  -0.833594  ,  -1.95628675,
       -17.95900934,   9.90341883, -27.00638292,   1.52930107,
       -15.58209462,   3.24977217,  10.61467642,   3.12386513,
         9.69583218,  13.45469613, -11.74664573,   0.83810425,
        -4.41190443, -15.03683097,   9.22118708, -20.97075208,
         3.49007006, -14.55346951,   3.69827719,   9.56741389,
         7.553959  ,   6.55940766,  15.04428306,  -8.24775089,
       -13.85182391,  -2.68081226,  -3.4921645 ,  -4.08495373,
       -15.79572271,  -6.63121145,  -5.27985443,  14.2791161 ,
        -0.48869955,   7.15436944,   5.7909795 ,  17.0585126 ,
         3.37211779,  -8.40272765, -11.95630534, -20.49241542,
       -21.76964907,   0.52262036,  -7.27174821, -12.46115223,
         0.77876468,  -9.24232837, -10.0308377 ,  -1.58369997,
        17.79648597,   3.37324026,  -7.41723784, -13.87993555,
       -32.35063997, -29.02041881,   2.75402301, -10.87102664,
       -14.00894147,  -0.10761005, -13.75258763,  -6.92

In [11]:
homero[1]

array([ 13.45469613, -11.74664573,   0.83810425,  -4.41190443,
       -15.03683097,   9.22118708, -20.97075208,   3.49007006,
       -14.55346951,   3.69827719,   9.56741389,   7.553959  ,
         6.55940766,  15.04428306,  -8.24775089, -13.85182391,
        -2.68081226,  -3.4921645 ,  -4.08495373, -15.79572271,
        -6.63121145,  -5.27985443,  14.2791161 ,  -0.48869955,
         7.15436944,   5.7909795 ,  17.0585126 ,   3.37211779,
        -8.40272765, -11.95630534, -20.49241542, -21.76964907,
         0.52262036,  -7.27174821, -12.46115223,   0.77876468,
        -9.24232837, -10.0308377 ,  -1.58369997,  17.79648597,
         3.37324026,  -7.41723784, -13.87993555, -32.35063997,
       -29.02041881,   2.75402301, -10.87102664, -14.00894147,
        -0.10761005, -13.75258763,  -6.92901847,   6.06855987,
        18.24939875,   4.51582829,  -1.49475212, -15.7522511 ,
       -42.73860277, -40.5503659 ,  16.14492621, -11.53843822,
       -18.72406055,  -0.19541117,  -0.66140254,  -1.14

In [12]:
homero_labels=np.array([[1,0]]*homero.shape[0])

In [13]:
homero_labels.shape

(1835, 2)

In [14]:
mundo_labels.shape

(8999, 2)

In [15]:
train_labels=np.vstack([homero_labels,mundo_labels])

In [16]:
train_set=np.vstack([homero,mundo])

In [17]:
train_set.shape

(10834, 260)

In [18]:
train_labels.shape

(10834, 2)

In [19]:
mfcc_feat=mfcc_wav("mundo_test_1.wav")
mundo_size=mfcc_feat.shape[0]-WINDOW_SIZE+1
mundo=np.array([mfcc_feat[i:i+WINDOW_SIZE].reshape(1,WINDOW_SIZE*mfcc_feat.shape[1])[0] for i in range(mundo_size)])
mundo_labels=np.array([[0,1]]*mundo.shape[0])
mfcc_feat=mfcc_wav("homero_test.wav")
homero_size=mfcc_feat.shape[0]-WINDOW_SIZE+1
homero=np.array([mfcc_feat[i:i+WINDOW_SIZE].reshape(1,WINDOW_SIZE*mfcc_feat.shape[1])[0] for i in range(homero_size)])
homero_labels=np.array([[1,0]]*homero.shape[0])
test_labels=np.vstack([homero_labels,mundo_labels])
test_set=np.vstack([homero,mundo])

mfcc_feat=mfcc_wav("mundo_test_2.wav")
mundo_size=mfcc_feat.shape[0]-WINDOW_SIZE+1
mundo=np.array([mfcc_feat[i:i+WINDOW_SIZE].reshape(1,WINDOW_SIZE*mfcc_feat.shape[1])[0] for i in range(mundo_size)])
mundo_labels=np.array([[0,1]]*mundo.shape[0])
mfcc_feat=mfcc_wav("homero_test_2.wav")
homero_size=mfcc_feat.shape[0]-WINDOW_SIZE+1
homero=np.array([mfcc_feat[i:i+WINDOW_SIZE].reshape(1,WINDOW_SIZE*mfcc_feat.shape[1])[0] for i in range(homero_size)])
homero_labels=np.array([[1,0]]*homero.shape[0])
test2_labels=np.vstack([homero_labels,mundo_labels])
test2_set=np.vstack([homero,mundo])

In [20]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [21]:
import tensorflow as tf
import shutil
from time import time
from tensorboard import summary as summary_lib
logs_path="log_dir"
logs_path1="log_dir/train"
logs_path2="log_dir/test"
# Parameters
learning_rate = 0.01
training_epochs = 42
batch_size = 256
display_step = 1
hidden_units=50

# Network Parameters
n_input =  train_set.shape[1] 
n_classes = 2 # classes


with tf.name_scope("inputs"):
    # tf Graph input
    X = tf.placeholder("float", [None, n_input],name="X")
with tf.name_scope("labels"):
    Y = tf.placeholder("float", [None, n_classes],name="Y")

# Construct model
with tf.name_scope('Capa1'):
    # Model
    weights1= tf.Variable(tf.random_normal([n_input, hidden_units]),name="weights1")
    bias1= tf.Variable(tf.random_normal([hidden_units]),name="bias1")
    act1= tf.nn.sigmoid(tf.matmul(X,weights1)+bias1, name="activacion_1")

with tf.name_scope('Capa2'):
    # Model
    weights2= tf.Variable(tf.random_normal([hidden_units, n_classes]),name="weights2")
    bias2= tf.Variable(tf.random_normal([n_classes]),name="bias2")
    logits= tf.matmul(act1,weights2)+bias2

with tf.name_scope('Loss'):
# Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits, labels=Y),name="costo")
with tf.name_scope('BGD'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,name="optimizador")
    train_op = optimizer.minimize(loss_op)
with tf.name_scope('Accuracy'):
    # Accuracy
    #pred = tf.nn.softmax(logits) # Softmax
    acc_op = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    acc_op = tf.reduce_mean(tf.cast(acc_op, tf.float32),name="acc_red_mean")
    
# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("loss", loss_op)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc_op)
# Merge all summaries into a single op
tf.summary.histogram('histogram', weights1)
merged_summary_op = tf.summary.merge_all()

t0=time()
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path1, graph=sess.graph)
    summary_writer_test = tf.summary.FileWriter(logs_path2, graph=sess.graph)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(train_set.shape[0]/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = next_batch(batch_size,train_set,train_labels)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c= sess.run([train_op, loss_op], feed_dict={Y: batch_y,
                                                            X: batch_x})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            summary, test_cost,_ = sess.run([merged_summary_op,loss_op,acc_op],
                                  feed_dict={X: test_set, Y: test_labels})
            summary_writer_test.add_summary(summary, epoch)
            print("Epoch:", '%04d' % (epoch+1), "train loss={:.9f} crossval loss={:.9f}".format(avg_cost,test_cost))
            summary, test_cost,_ = sess.run([merged_summary_op,loss_op,acc_op],
                                  feed_dict={X: train_set, Y: train_labels})
            summary_writer.add_summary(summary, epoch)
    print("Optimization Finished")        

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy homero1:", accuracy.eval({X: homero, Y: homero_labels})) 
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy mundo1:", accuracy.eval({X: mundo, Y: mundo_labels}))
print("done in %0.3fs." % (time() - t0))

/home/cselmo/anaconda3/envs/OpLaDyn/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Epoch: 0001 train loss=0.611430585 crossval loss=0.448574364
Epoch: 0002 train loss=0.276947543 crossval loss=0.285371989
Epoch: 0003 train loss=0.238403237 crossval loss=0.331320584
Epoch: 0004 train loss=0.207221505 crossval loss=0.342725068
Epoch: 0005 train loss=0.184399819 crossval loss=0.305889577
Epoch: 0006 train loss=0.176423017 crossval loss=0.305490494
Epoch: 0007 train loss=0.157909141 crossval loss=0.298329502
Epoch: 0008 train loss=0.162333765 crossval loss=0.282848418
Epoch: 0009 train loss=0.141929320 crossval loss=0.285952210
Epoch: 0010 train loss=0.137945246 crossval loss=0.295814604
Epoch: 0011 train loss=0.147513849 crossval loss=0.283862621
Epoch: 0012 train loss=0.135579473 crossval loss=0.311871678
Epoch: 0013 train loss=0.137076363 crossval loss=0.318696499
Epoch: 0014 train loss=0.129753120 crossval loss=0.235178530
Epoch: 0015 train loss=0.117921015 crossval loss=0.276717901
Epoch: 0016 train loss=0.112030178 crossval loss=0.295880616
Epoch: 0017 train loss=0